# 5. 분류 (classification)
분류는 대상의 속성을 입력 받고, 목표 변수가 갖고 있는 카테고리(범주형) 값 중에서 분류하여 예측한다. 목표 변수 값을 함께 입력하기 때문에 지도 학습 유형에 속하는 알고리즘이다.  
KNN, SVM, Decision Tree, Logistic Regression 등의 알고리즘이 존재한다.

## KNN (k-Nearest-Neighbors)
KNN은 k개의 가까운 이윳이라는 뜻이다. 관측값이 주어지면 관측값을 기준으로 가까운 순서로 k개의 속성을 찾고, 가장 많은 속성으로 분류한다.  
이때 k값에 따라 정확도가 달라지므로 적절한 k값을 찾는 것이 매우 중요하다.

데이터는 Seaborn 라이브러리에서 'titanic'데이터 셋을 활용한다.

In [1]:
import pandas as pd
import seaborn as sns
df = sns.load_dataset('titanic')
print(df.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


In [2]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None


데이터에서 NaN 값이 많은 열을 삭제하고 전처리를 진행하자.

In [3]:
rdf = df.drop(['deck','embark_town'], axis=1) # axis=1 열 기준 연산
print(rdf.columns.values)

['survived' 'pclass' 'sex' 'age' 'sibsp' 'parch' 'fare' 'embarked' 'class'
 'who' 'adult_male' 'alive' 'alone']


In [5]:
rdf = rdf.dropna(subset=['age'], how='any', axis=0)

most_freq = rdf['embarked'].value_counts(dropna=True).idxmax() # print(rdf.describe(include='all'))
rdf['embarked'].fillna(most_freq, inplace=True)

분석에 사용할 속성을 지정하고, get_dummies를 활용하여 원핫인코딩을 적용하고 prefix 옵션을 사용하여 접두어를 붙여주자.  
##### Q1. 원핫인코딩이 뭐지
- onehot
- get_dummies
- prefix

In [7]:
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]

onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1) #axis=1 : 열 기준 연산

ndf.drop(['sex','embarked'], axis=1, inplace=True)

독립변수를 지정하고  
StandardScaler()preprocessing.StandardScalar().fit(X).transform(X)을 활용하여  
데이터의 상대적 크기 차이를 없애기 위하여 데이터 정규화(normalization)을 진행한다.

In [9]:
X=ndf[['pclass','age','sibsp','parch','female','male',
      'town_C','town_Q','town_S']] #독립 변수 X
y=ndf['survived']

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X) # standardScaler => 통계화 시킴.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3,
                                                    random_state=10)
# 트레이닝 데이터와 테스트 데이터를 split으로 분리

사이킷런에서 KNeighborsClassifier를 통해 KNN 분류 모형을 가져오자. 이때 k값을 넣어준다. 비지도라 k값은 알 수가 없고 여러개 해보고 제일 좋은 것을 선택한다. 그리고 학습을 진행하고 예측값과 결과값을 비교해 보자.

In [10]:
from sklearn.neighbors import KNeighborsClassifier #최 근접 분류모델을 쓴다.

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_hat = knn.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 1 0 0 1 1 1 0 0]
[0 0 1 0 0 1 1 1 0 0]


모형의 예측 능력을 평가해 보자. metrics 모듈의 confusion_matrix()함수를 사용하여 Confusion Matrix를 계산하자. 그리고 classification_report() 함수를 사용하여 precision, recall, f1-score 지표를 출력해준다. 관련 개념은 코드 아래 부분을 참고하자.

In [11]:
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat) # 매트릭스 => 평가 측정 지표
print(knn_matrix)

knn_report = metrics.classification_report(y_test, y_hat) #정리해서 도출되기 때문에 R보다 좋다
print(knn_report) # confusion_matrix는 헷갈리고 시험도 많이 나온다.

[[109  16]
 [ 25  65]]
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       125
           1       0.80      0.72      0.76        90

    accuracy                           0.81       215
   macro avg       0.81      0.80      0.80       215
weighted avg       0.81      0.81      0.81       215



## 분류 모형의 예측력을 평가하는 지표

1. Confusion Matrix
2. Precision (정확도)
3. Recall (재현율)
4. F1-score (F1 지표)

### 1. Confusion Matrix
모형을 예측하는 값에는 True와 False가 있다. 그리고 아래의 그림과 같이 모형의 예측값과 실제 값을 각각 축으로 하는 2x2 매트릭스로 표현한 것을 Confusion Matrix라고 부른다.

|예측값|||
|:------:|:---:|:---:|
|T|TP (True Positive)|FP (False Positive)|
|F|FN (False Negative)|TN (True Negative)|
|실제값|T|F|

### 2. 정확도 (Precision)
True로 예측한 분석대상 중에서 실제 값이 True인 비율을 말하며, 모형의 정확성을 나타내는 지표가 된다. 정확도가 높다는 것은 False Positive(실제 False를 True로 잘못 예측) 오류가 적다는 말이다.

$$ Precision = \frac{TP}{TP+FP}$$

### 3. 재현율 (Recall)
실제 값이 True인 분석대상 중에서 True로 예측하여 모형이 적중한 비율을 말하며, 모형의 완전성을 나타내는 지표이다.  
재현율이 높다는 것은 False Negative(실제 True를 False로 잘못 예측) 오류가 낮다는 뜻이다.

$$ Recall = \frac{TP}{TP+FN} $$

### 4. F1 지표 (F1-score)
정확도와 재현율이 균등하게 반영될 수 있도록 정확도와 재현율의 조화평균을 계산한 값으로, 모형의 예측력을 종합적으로 평가하는 지표이다.  
값이 높을수록 분류 모형의 예측력이 좋다고 말할 수 있다.

$$ F1 score = 2*\frac{Precision*Recall}{Precision+Recall}$$

In [18]:
from sklearn.neighbors import KNeighborsClassifier #최 근접 분류모델을 쓴다.

knn = KNeighborsClassifier(n_neighbors=19) # k값
# k 값이 크다도 무작정 좋은 것이 아닌거 같다. 그냥 여러가지를 해보고 가장 적당한 것을 결과로 뽑는것이 좋다.
knn.fit(X_train, y_train)
y_hat = knn.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat) # 매트릭스 => 평가 측정 지표
print(knn_matrix)

knn_report = metrics.classification_report(y_test, y_hat) #정리해서 도출되기 때문에 R보다 좋다
print(knn_report) # confusion_matrix는 헷갈리고 시험도 많이 나온다.

[0 0 1 0 0 0 1 0 1 0]
[0 0 1 0 0 1 1 1 0 0]
[[118   7]
 [ 38  52]]
              precision    recall  f1-score   support

           0       0.76      0.94      0.84       125
           1       0.88      0.58      0.70        90

    accuracy                           0.79       215
   macro avg       0.82      0.76      0.77       215
weighted avg       0.81      0.79      0.78       215



In [24]:
from sklearn.neighbors import KNeighborsClassifier #최 근접 분류모델을 쓴다.
from sklearn import metrics

a = int(input('insert cycle: '))

for i in range(1,a):
    knn[0] = 0
    knn[i] = KNeighborsClassifier(n_neighbors=i) # k값
    

    knn[i].fit(X_train, y_train)
    y_hat = knn[i].predict(X_test)

    print(y_hat[0:10])
    print(y_test.values[0:10])


    knn_matrix[i] = metrics.confusion_matrix(y_test, y_hat) # 매트릭스 => 평가 측정 지표
    a = knn_matrix[i].sum()
    b = knn_matrix[i+1].sum()
    if knn_matrix[i].sum() > knn_matrix[i+1].sum():
        print(knn_matrix[i])
    else:
        print(knn_matrix[i+1])

    knn_report = metrics.classification_report(y_test, y_hat) #정리해서 도출되기 때문에 R보다 좋다
    print(knn_report) # confusion_matrix는 헷갈리고 시험도 많이 나온다.

insert cycle: 5


TypeError: 'KNeighborsClassifier' object does not support item assignment